### Descripción del Nuevo Dataset
Generaremos un dataset que simule datos de sensores en una fábrica. Cada registro representa una lectura de sensores en diferentes estaciones de trabajo, con las siguientes variables:
- **ID de la estación**: identificador único de cada estación.
- **Temperatura**: lectura de la temperatura en grados Celsius.
- **Humedad**: porcentaje de humedad.
- **Vibración**: nivel de vibración medido (escala del 1 al 100).
- **Fecha de lectura**: año de la lectura.

Utilizaremos las funciones restantes de PyTorch en este contexto y explicaré su uso en cada paso.

### Generación del Dataset Artificial

```python
import torch

# Configuración de las semillas para reproducibilidad
torch.manual_seed(42)

# Número de registros simulados
num_records = 200

# Generación de datos de las columnas
station_ids = torch.randint(1, 21, (num_records,))  # 20 estaciones únicas
temperatures = torch.randint(15, 45, (num_records,)).float()  # Temperaturas entre 15 y 45 °C
humidity = torch.randint(30, 80, (num_records,)).float()  # Humedad entre 30% y 80%
vibrations = torch.randint(1, 101, (num_records,))  # Vibración entre 1 y 100
reading_years = torch.randint(2019, 2024, (num_records,))  # Lecturas entre 2019 y 2023

# Creación del dataset en forma de tensor
sensor_data = torch.stack((station_ids, temperatures, humidity, vibrations, reading_years), dim=1)
print("Dataset de sensores:\n", sensor_data[:10])  # Muestra las primeras 10 filas
```

### Análisis Descriptivo Completo Usando Comandos No Cubiertos Anteriormente

#### 1. Reducción y Acumulación con `torch.prod()`, `torch.cumsum()`, y `torch.cumprod()`
Estas funciones nos ayudan a realizar operaciones acumulativas o de producto en un tensor.

1. **Producto de Vibraciones**: Calculamos el producto de todas las lecturas de vibración.
   ```python
   vibration_prod = torch.prod(sensor_data[:, 3].float())
   print("Producto de todas las lecturas de vibración:", vibration_prod)
   ```

2. **Suma Acumulativa de Temperaturas**: Calculamos la suma acumulativa de las lecturas de temperatura.
   ```python
   temperature_cumsum = torch.cumsum(sensor_data[:, 1], dim=0)
   print("Suma acumulativa de temperaturas:", temperature_cumsum[:10])  # Mostrar las primeras 10
   ```

3. **Producto Acumulativo de Humedad**: Calculamos el producto acumulativo de las lecturas de humedad.
   ```python
   humidity_cumprod = torch.cumprod(sensor_data[:, 2], dim=0)
   print("Producto acumulativo de humedad:", humidity_cumprod[:10])  # Mostrar las primeras 10
   ```

#### 2. Identificación de Mínimos y Máximos con `torch.argmin()` y `torch.argmax()`
Estas funciones nos devuelven el índice de los valores mínimo y máximo en un tensor.

1. **Índice de Temperatura Mínima y Máxima**: Identificamos los índices de la temperatura más baja y más alta.
   ```python
   temp_min_index = torch.argmin(sensor_data[:, 1])
   temp_max_index = torch.argmax(sensor_data[:, 1])
   print(f"Índice de temperatura mínima: {temp_min_index}, Temperatura mínima: {sensor_data[temp_min_index, 1]}")
   print(f"Índice de temperatura máxima: {temp_max_index}, Temperatura máxima: {sensor_data[temp_max_index, 1]}")
   ```

#### 3. Operaciones Lógicas y Filtrado Condicional con `torch.where()` y `torch.masked_select()`
Estas funciones nos permiten realizar selecciones condicionales en un tensor.

1. **Estaciones con Temperatura Mayor a 40 °C**: Usamos `torch.where()` para identificar estaciones con temperaturas peligrosamente altas.
   ```python
   high_temp = torch.where(sensor_data[:, 1] > 40, sensor_data[:, 1], torch.tensor(float('nan')))
   print("Temperaturas mayores a 40 °C:", high_temp[~torch.isnan(high_temp)])  # Filtra NaN
   ```

2. **Filtrado de Vibración Alta**: Usamos `torch.masked_select()` para seleccionar registros con vibración mayor a 80.
   ```python
   high_vibration_mask = sensor_data[:, 3] > 80
   high_vibrations = torch.masked_select(sensor_data[:, 3], high_vibration_mask)
   print("Lecturas de vibración mayores a 80:", high_vibrations)
   ```

#### 4. Transformación de Forma y Manipulación con `torch.reshape()`, `torch.view()`, `torch.transpose()`, `torch.permute()`
Estas funciones nos permiten reorganizar las dimensiones del tensor.

1. **Reorganización para Visualización**: Usamos `reshape` para dividir el dataset en bloques de 50 registros para un análisis por lote.
   ```python
   reshaped_data = sensor_data.reshape(4, 50, 5)  # 4 bloques de 50 registros cada uno
   print("Datos reorganizados en bloques de 50:\n", reshaped_data[0])  # Mostrar el primer bloque
   ```

2. **Visualización de Transposición y Permutación**: Transponemos y permutamos el tensor para que las columnas se conviertan en filas y viceversa.
   ```python
   transposed_data = torch.transpose(sensor_data, 0, 1)  # Cambia filas por columnas
   print("Dataset transpuesto:\n", transposed_data)
   
   permuted_data = sensor_data.permute(1, 0)  # Alternativa a transpose
   print("Dataset permutado:\n", permuted_data)
   ```

#### 5. Conversión de Tipos y Expansión con `expand()` y `expand_as()`
Estas funciones son útiles para ajustar tipos de datos y expandir tensores para que coincidan en operaciones de broadcasting.

1. **Conversión de Tipos**: Cambiamos la temperatura y la humedad a `float32` para operaciones más precisas.
   ```python
   temperatures_float = sensor_data[:, 1].float()
   humidity_float = sensor_data[:, 2].float()
   ```

2. **Expansión para Cálculo por Año**: Usamos `expand()` para calcular el promedio de vibración por cada año de lectura.
   ```python
   unique_years = torch.unique(sensor_data[:, 4])
   expanded_years = unique_years.expand(num_records, unique_years.size(0))
   print("Años expandidos para análisis:", expanded_years[:10])
   ```

#### 6. Uso de Autograd con `requires_grad_()` y `backward()`
Aunque normalmente se aplica en redes neuronales, `autograd` permite calcular derivadas. Aquí simularemos un cálculo donde una variable depende de otra y necesitamos el gradiente.

1. **Simulación de Derivadas**: Calculamos la derivada de una función de temperatura respecto a la vibración.
   ```python
   temperatures = sensor_data[:, 1].float().requires_grad_()
   vibrations = sensor_data[:, 3].float()

   # Función simulada de dependencia: temperatura = f(vibración)
   loss = torch.sum(temperatures * vibrations)  # Función de pérdida ficticia
   loss.backward()  # Calcula el gradiente
   print("Gradiente de temperatura:", temperatures.grad)
   ```